In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
import xgboost as xgb
import random

In [2]:
data = pd.read_csv("minute_data/BTC-USD_1M_SIGNALS.csv",)

In [3]:
data

,Unix,Date,Symbol,Open,High,Low,Close,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14
0,1483228860,2017-01-01 00:01:00,BTC/USD,966.34,966.34,966.34,966.34,-0.312459,-0.006352,-0.291721,0.003268,0.001095,-0.296745,-0.163178,-0.618641
1,1483228920,2017-01-01 00:02:00,BTC/USD,966.16,966.37,966.16,966.37,-0.310186,-0.000855,0.828292,0.003534,0.002246,-0.285351,-0.163178,-0.668323
2,1483228980,2017-01-01 00:03:00,BTC/USD,966.37,966.37,966.37,966.37,-0.348678,-0.000855,0.828292,0.003738,0.003484,-0.296745,-0.163178,-0.647506
3,1483229040,2017-01-01 00:04:00,BTC/USD,966.37,966.37,966.37,966.37,-0.348678,-0.011849,0.835669,0.003932,0.004996,-0.293490,-0.163178,-0.649919
4,1483229100,2017-01-01 00:05:00,BTC/USD,966.43,966.43,966.43,966.43,-0.348169,-0.028336,0.910591,0.004110,0.006831,-0.288606,-0.163178,-0.652332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2674991,1646105880,2022-03-01 03:38:00,BTC/USD,43077.82,43077.82,43049.46,43049.46,-0.343969,0.205143,3.243746,202.140130,315.950658,-0.296745,1.189788,0.570316
2674992,1646105940,2022-03-01 03:39:00,BTC/USD,43035.16,43035.16,42999.44,42999.44,-0.174815,-0.069151,3.243036,182.098937,328.738652,0.603408,-0.163178,0.176675
2674993,1646106000,2022-03-01 03:40:00,BTC/USD,43022.24,43022.24,43016.03,43016.03,-0.346723,-0.126531,3.243035,153.986631,318.063305,-0.177376,-0.163178,-0.189747
2674994,1646106060,2022-03-01 03:41:00,BTC/USD,43018.23,43046.59,43018.23,43046.58,-0.318365,-0.000855,3.243946,115.826724,271.530088,1.241492,-0.163178,-0.539146


In [4]:
data[0:20]

,Unix,Date,Symbol,Open,High,Low,Close,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14
0,1483228860,2017-01-01 00:01:00,BTC/USD,966.34,966.34,966.34,966.34,-0.312459,-0.006352,-0.291721,0.003268,0.001095,-0.296745,-0.163178,-0.618641
1,1483228920,2017-01-01 00:02:00,BTC/USD,966.16,966.37,966.16,966.37,-0.310186,-0.000855,0.828292,0.003534,0.002246,-0.285351,-0.163178,-0.668323
2,1483228980,2017-01-01 00:03:00,BTC/USD,966.37,966.37,966.37,966.37,-0.348678,-0.000855,0.828292,0.003738,0.003484,-0.296745,-0.163178,-0.647506
3,1483229040,2017-01-01 00:04:00,BTC/USD,966.37,966.37,966.37,966.37,-0.348678,-0.011849,0.835669,0.003932,0.004996,-0.293490,-0.163178,-0.649919
4,1483229100,2017-01-01 00:05:00,BTC/USD,966.43,966.43,966.43,966.43,-0.348169,-0.028336,0.910591,0.004110,0.006831,-0.288606,-0.163178,-0.652332
5,1483229160,2017-01-01 00:06:00,BTC/USD,966.58,966.58,966.58,966.58,-0.347099,-0.000855,1.110668,0.004124,0.008586,-0.296745,-0.163178,-0.665621
6,1483229220,2017-01-01 00:07:00,BTC/USD,966.58,966.58,966.58,966.58,-0.348678,-0.000855,1.110668,0.003720,0.009398,-0.296745,-0.163178,-0.721027
7,1483229280,2017-01-01 00:08:00,BTC/USD,966.58,966.58,966.58,966.58,-0.348678,-0.000855,0.063198,0.003137,0.009823,-0.296745,-0.163178,-0.740746
8,1483229340,2017-01-01 00:09:00,BTC/USD,966.58,966.58,966.58,966.58,-0.348678,0.000977,0.063198,0.002335,0.009628,-0.296745,-0.162861,-0.762462
9,1483229400,2017-01-01 00:10:00,BTC/USD,966.57,966.57,966.57,966.57,-0.345367,0.022965,0.987989,0.001263,0.008497,-0.296745,-0.163178,-0.786936


In [5]:
predict_data = data.copy().drop(["Open","Close","High","Low", "Symbol"],axis=1)
#max_days = 5
#target_range = 2
max_days = 20
target_range = 6
for i in range(1,max_days):#2jours
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG"]].shift(i)
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i), "ADX-{}".format(i), "+DM-{}".format(i), "-DM-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG","ADX14","+DM","-DM"]].shift(i)
    predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i)]] = data[["Variation","Volume USD","RSI","MACD","MACD_H"]].shift(i)
#predict_data["Target"] = (data["Variation"].shift(-1) >= 0)
predict_data["Target"] = (data["Close"].shift(-target_range) - data["Close"] >= 0)
predict_data["Target"] = np.where(predict_data["Target"] == True, 1, 0)
predict_data.dropna(inplace=True)
predict_data.reset_index(inplace=True,drop=True)
predict_data = predict_data[0:len(predict_data)-target_range]

In [6]:
predict_data

,Unix,Date,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,Vol-1,RSI-1,MACD-1,MACD_H-1,Variation-2,Vol-2,RSI-2,MACD-2,MACD_H-2,Variation-3,Vol-3,RSI-3,MACD-3,MACD_H-3,Variation-4,Vol-4,RSI-4,MACD-4,MACD_H-4,Variation-5,Vol-5,RSI-5,MACD-5,MACD_H-5,Variation-6,Vol-6,RSI-6,MACD-6,MACD_H-6,...,Vol-12,RSI-12,MACD-12,MACD_H-12,Variation-13,Vol-13,RSI-13,MACD-13,MACD_H-13,Variation-14,Vol-14,RSI-14,MACD-14,MACD_H-14,Variation-15,Vol-15,RSI-15,MACD-15,MACD_H-15,Variation-16,Vol-16,RSI-16,MACD-16,MACD_H-16,Variation-17,Vol-17,RSI-17,MACD-17,MACD_H-17,Variation-18,Vol-18,RSI-18,MACD-18,MACD_H-18,Variation-19,Vol-19,RSI-19,MACD-19,MACD_H-19,Target
0,1483230000,2017-01-01 00:20:00,-0.348678,-0.000855,0.361261,-0.003473,0.005520,-0.296745,-0.163178,1.915589,-0.000855,-0.348360,0.361261,-0.003338,0.004772,-0.000855,-0.347010,0.361261,-0.003205,0.004169,0.000979,-0.348678,0.370753,-0.003056,0.003729,-0.000855,-0.346988,0.370753,-0.002915,0.003356,0.184380,-0.253466,1.087972,-0.000942,0.007898,-0.187728,-0.348174,0.497719,-0.001253,0.005496,...,-0.348678,0.063198,0.003137,0.009823,-0.000855,-0.348678,1.110668,0.003720,0.009398,-0.000855,-0.347099,1.110668,0.004124,0.008586,-0.028336,-0.348169,0.910591,0.004110,0.006831,-0.011849,-0.348678,0.835669,0.003932,0.004996,-0.000855,-0.348678,0.828292,0.003738,0.003484,-0.000855,-0.310186,0.828292,0.003534,0.002246,-0.006352,-0.312459,-0.291721,0.003268,0.001095,0
1,1483230060,2017-01-01 00:21:00,-0.348678,-0.182238,-0.601917,-0.003607,0.006457,-0.253881,-0.163178,2.285288,-0.000855,-0.348678,0.361261,-0.003473,0.005520,-0.000855,-0.348360,0.361261,-0.003338,0.004772,-0.000855,-0.347010,0.361261,-0.003205,0.004169,0.000979,-0.348678,0.370753,-0.003056,0.003729,-0.000855,-0.346988,0.370753,-0.002915,0.003356,0.184380,-0.253466,1.087972,-0.000942,0.007898,...,-0.348678,0.063198,0.002335,0.009628,-0.000855,-0.348678,0.063198,0.003137,0.009823,-0.000855,-0.348678,1.110668,0.003720,0.009398,-0.000855,-0.347099,1.110668,0.004124,0.008586,-0.028336,-0.348169,0.910591,0.004110,0.006831,-0.011849,-0.348678,0.835669,0.003932,0.004996,-0.000855,-0.348678,0.828292,0.003738,0.003484,-0.000855,-0.310186,0.828292,0.003534,0.002246,0
2,1483230120,2017-01-01 00:22:00,-0.347836,-0.000855,0.204509,-0.003739,0.007635,-0.285893,-0.163178,2.614295,-0.182238,-0.348678,-0.601917,-0.003607,0.006457,-0.000855,-0.348678,0.361261,-0.003473,0.005520,-0.000855,-0.348360,0.361261,-0.003338,0.004772,-0.000855,-0.347010,0.361261,-0.003205,0.004169,0.000979,-0.348678,0.370753,-0.003056,0.003729,-0.000855,-0.346988,0.370753,-0.002915,0.003356,...,-0.345367,0.987989,0.001263,0.008497,0.000977,-0.348678,0.063198,0.002335,0.009628,-0.000855,-0.348678,0.063198,0.003137,0.009823,-0.000855,-0.348678,1.110668,0.003720,0.009398,-0.000855,-0.347099,1.110668,0.004124,0.008586,-0.028336,-0.348169,0.910591,0.004110,0.006831,-0.011849,-0.348678,0.835669,0.003932,0.004996,-0.000855,-0.348678,0.828292,0.003738,0.003484,0
3,1483230180,2017-01-01 00:23:00,-0.348521,0.305644,0.172745,-0.006168,0.001522,-0.296745,-0.163178,3.024138,-0.000855,-0.347836,0.204509,-0.003739,0.007635,-0.182238,-0.348678,-0.601917,-0.003607,0.006457,-0.000855,-0.348678,0.361261,-0.003473,0.005520,-0.000855,-0.348360,0.361261,-0.003338,0.004772,-0.000855,-0.347010,0.361261,-0.003205,0.004169,0.000979,-0.348678,0.370753,-0.003056,0.003729,...,-0.342565,0.935494,-0.000112,0.006078,0.022965,-0.345367,0.987989,0.001263,0.008497,0.000977,-0.348678,0.063198,0.002335,0.009628,-0.000855,-0.348678,0.063198,0.003137,0.009823,-0.000855,-0.348678,1.110668,0.003720,0.009398,-0.000855,-0.347099,1.110668,0.004124,0.008586,-0.028336,-0.348169,0.910591,0.004110,0.006831,-0.011849,-0.348678,0.835669,0.003932,0.004996,0
4,1483230240,2017-01-01 00:24:00,-0.346488,-0.000855,-3.145265,-0.009191,-0.008083,-0.296745,-0.110214,3.413551,0.305644,-0.348521,0.172745,-0.006168,0.001522,-0.000855,-0.347836,0.204509,-0.003739,0.007635,-0.182238,-0.348678,-0.601917,-0.003607,0.006457,-0.000855

In [7]:
predict_data = predict_data[[i % int(max_days) == 0 for i in range(len(predict_data))]]

In [8]:
predict_data

,Unix,Date,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,Vol-1,RSI-1,MACD-1,MACD_H-1,Variation-2,Vol-2,RSI-2,MACD-2,MACD_H-2,Variation-3,Vol-3,RSI-3,MACD-3,MACD_H-3,Variation-4,Vol-4,RSI-4,MACD-4,MACD_H-4,Variation-5,Vol-5,RSI-5,MACD-5,MACD_H-5,Variation-6,Vol-6,RSI-6,MACD-6,MACD_H-6,...,Vol-12,RSI-12,MACD-12,MACD_H-12,Variation-13,Vol-13,RSI-13,MACD-13,MACD_H-13,Variation-14,Vol-14,RSI-14,MACD-14,MACD_H-14,Variation-15,Vol-15,RSI-15,MACD-15,MACD_H-15,Variation-16,Vol-16,RSI-16,MACD-16,MACD_H-16,Variation-17,Vol-17,RSI-17,MACD-17,MACD_H-17,Variation-18,Vol-18,RSI-18,MACD-18,MACD_H-18,Variation-19,Vol-19,RSI-19,MACD-19,MACD_H-19,Target
0,1483230000,2017-01-01 00:20:00,-0.348678,-0.000855,0.361261,-0.003473,0.005520,-0.296745,-0.163178,1.915589,-0.000855,-0.348360,0.361261,-0.003338,0.004772,-0.000855,-0.347010,0.361261,-0.003205,0.004169,0.000979,-0.348678,0.370753,-0.003056,0.003729,-0.000855,-0.346988,0.370753,-0.002915,0.003356,0.184380,-0.253466,1.087972,-0.000942,0.007898,-0.187728,-0.348174,0.497719,-0.001253,0.005496,...,-0.348678,0.063198,0.003137,0.009823,-0.000855,-0.348678,1.110668,0.003720,0.009398,-0.000855,-0.347099,1.110668,0.004124,0.008586,-0.028336,-0.348169,0.910591,0.004110,0.006831,-0.011849,-0.348678,0.835669,0.003932,0.004996,-0.000855,-0.348678,0.828292,0.003738,0.003484,-0.000855,-0.310186,0.828292,0.003534,0.002246,-0.006352,-0.312459,-0.291721,0.003268,0.001095,0
20,1483231200,2017-01-01 00:40:00,-0.324111,-0.000855,-1.199659,0.003195,-0.010350,-0.296203,-0.163178,0.901176,-0.002687,-0.348678,-1.212909,0.002775,-0.009390,-0.000855,-0.346876,-1.212909,0.002404,-0.008493,0.000977,-0.324070,-1.183371,0.002094,-0.007614,-0.006352,-0.348678,-1.222406,0.001762,-0.006968,-0.000855,-0.347903,-1.222406,0.001472,-0.006342,-0.207924,-0.348678,-2.053923,-0.000840,-0.011184,...,-0.348678,-3.104569,-0.007500,-0.012720,-0.000855,-0.348678,-3.145534,-0.008245,-0.012144,-0.000855,-0.348678,-3.145534,-0.008745,-0.011036,-0.000855,-0.348678,-3.145265,-0.009047,-0.009628,-0.000855,-0.346488,-3.145265,-0.009191,-0.008083,0.305644,-0.348521,0.172745,-0.006168,0.001522,-0.000855,-0.347836,0.204509,-0.003739,0.007635,-0.182238,-0.348678,-0.601917,-0.003607,0.006457,0
40,1483232400,2017-01-01 01:00:00,-0.348121,-0.000855,3.231226,0.017293,0.011716,-0.296745,-0.163178,0.183850,-0.000855,-0.348678,3.225409,0.017017,0.008644,-0.000855,-0.348388,3.225409,0.016597,0.005804,0.070594,-0.348678,3.226101,0.016778,0.005123,-0.000855,-0.347980,3.225686,0.016719,0.003942,-0.004518,-0.348508,3.148116,0.016436,0.002404,-0.000855,-0.348512,2.798183,0.016016,0.000813,...,-0.250296,2.933221,0.012173,-0.006239,-0.105241,-0.348107,-0.625977,0.010435,-0.009584,-0.074152,-0.305502,-1.368542,0.008215,-0.013532,-0.011854,-0.324796,-1.444365,0.006264,-0.015984,0.079810,-0.348402,-0.345761,0.005454,-0.014927,0.000978,-0.348678,-1.255790,0.004765,-0.013761,-0.000855,-0.324111,-1.255790,0.004162,-0.012605,0.000977,-0.348678,-1.199659,0.003649,-0.011439,1
60,1483233600,2017-01-01 01:20:00,-0.347768,-0.000855,-0.482859,-0.002237,-0.001158,-0.296745,-0.163178,-1.050364,0.275015,-0.348507,0.595833,-0.000612,0.003368,0.017508,-0.348588,0.653164,0.000840,0.006531,0.100130,-0.348678,0.897643,0.002959,0.010825,-0.000855,-0.347970,1.340361,0.004576,0.012932,-0.455944,-0.348678,-0.108854,0.001264,0.001592,-0.000855,-0.206456,-0.108854,-0.001356,-0.005649,...,-0.305793,0.712666,0.006572,0.017188,-0.000855,-0.348678,1.412501,0.010090,0.023048,-0.000855,-0.348678,1.412501,0.012722,0.025392,-0.000855,-0.348250,1.029192,0.014628,0.025351,-0.000855,-0.348459,0.997911,0.015945,0.023760,-0.002687,-0.347933,0.802704,0.016766,0.021179,0.000977,-0.348678,0.806421,0.017227,0.018159,-0.000855,-0.348678,3.231226,0.017381,0.014935,1
80,1483234800,2017-01-01 01:40:00,-0.348441,-0.000855,0.921279,-0.004101,0.005929,-0.296745,-0.163178,0.203604,-0.019222,-0.348678,0.701775,-0.002966,0.007742,-0.000855,-0.348678,0.701775,-0.002053,0.008605,-0.000855,-0.310693,0.83

In [9]:
#corr = predict_data.corr("pearson")
#corr[["Target"]].to_clipboard()
#corr[["Target"]]

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
split_mode = "RAND"

if split_mode == "RAND":
    train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.2, random_state=100)
elif split_mode == "STATIC":
    train_data = predict_data[0:int(0.66*len(predict_data))].drop(["Date", "Unix", "Target"],axis=1)
    train_labels = predict_data[0:int(0.66*len(predict_data))]["Target"]
    test_data = predict_data[int(0.66*len(predict_data)):len(predict_data)].drop(["Date", "Unix", "Target"],axis=1)
    test_labels = predict_data[int(0.66*len(predict_data)):len(predict_data)]["Target"]

In [12]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [12]:
algo = "MLP"

if algo == "MLP":
    model = MLPClassifier(hidden_layer_sizes=(100,200,100),verbose=True)
elif algo == "RF":
    model = RandomForestClassifier(n_estimators=100,verbose=False)
elif algo == "CAT":
    model = CatBoostClassifier(iterations=100,depth=12)
elif algo == "TREE":
    model = DecisionTreeClassifier(splitter="random")
model.fit(train_data,train_labels)

Iteration 1, loss = 0.65398021
Iteration 2, loss = 0.64336664
Iteration 3, loss = 0.63996147
Iteration 4, loss = 0.63833888
Iteration 5, loss = 0.63697008
Iteration 6, loss = 0.63595846
Iteration 7, loss = 0.63490123
Iteration 8, loss = 0.63386533
Iteration 9, loss = 0.63308808
Iteration 10, loss = 0.63230100
Iteration 11, loss = 0.63149277
Iteration 12, loss = 0.63073962
Iteration 13, loss = 0.62990224
Iteration 14, loss = 0.62882980
Iteration 15, loss = 0.62831250
Iteration 16, loss = 0.62773880
Iteration 17, loss = 0.62650033
Iteration 18, loss = 0.62577357
Iteration 19, loss = 0.62499218
Iteration 20, loss = 0.62419966
Iteration 21, loss = 0.62347343
Iteration 22, loss = 0.62242316
Iteration 23, loss = 0.62162843
Iteration 24, loss = 0.62097936
Iteration 25, loss = 0.61968942
Iteration 26, loss = 0.61909913
Iteration 27, loss = 0.61817302
Iteration 28, loss = 0.61718370
Iteration 29, loss = 0.61618977
Iteration 30, loss = 0.61551841
Iteration 31, loss = 0.61454742
Iteration 32, los

c:\Users\Alexis\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(hidden_layer_sizes=(100, 200, 100), verbose=True)

In [50]:
model.score(test_data,test_labels)

0.5018050541516246

In [51]:
model.score(train_data,train_labels)

1.0

In [35]:
from sklearn.metrics import confusion_matrix

In [53]:
preds = model.predict(test_data)

In [54]:
conf = confusion_matrix(test_labels,preds)
conf

array([[53, 72],
       [66, 86]], dtype=int64)

In [55]:
TN = conf[0,0] / (conf[0,0] + conf[1,0])
TN

0.44537815126050423

In [56]:
TP = conf[1,1] / (conf[1,1] + conf[0,1])
TP

0.5443037974683544

## Cross Validation

In [19]:
def cross_validation_loop(predict_data, model_name):
    scores = []
    TPs = []
    TNs = []
    for randomize in range(1,20):
        train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.33, random_state=randomize)
        if model_name == "MLP":
            model = MLPClassifier(hidden_layer_sizes=(200,2000,200))
        elif model_name == "CAT":
            model = CatBoostClassifier(iterations=100,depth=12,verbose=False)
        elif model_name == "RF":
            model = RandomForestClassifier(n_estimators=100,verbose=False)
        model.fit(train_data, train_labels)
        scores.append(model.score(test_data, test_labels))
        conf = confusion_matrix(test_labels,model.predict(test_data))
        TN = conf[0,0] / (conf[0,0] + conf[1,0])
        TP = conf[1,1] / (conf[1,1] + conf[0,1])
        TPs.append(TP)
        TNs.append(TN)
        print("Score : {} || TP : {} || TN : {}".format(scores[-1],TP,TN))


    return scores,TPs,TNs

In [20]:
#scores,TPs,TNs = cross_validation_loop(predict_data,"CAT")

In [24]:
from statistics import mean

In [50]:
print("CatBoost score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

score moyen : 0.5067452023560707 || TP moyen : 0.5411620348700146 || TN moyen : 0.4619500842660272


In [22]:
scores,TPs,TNs = cross_validation_loop(predict_data,"RF")

Score : 0.51985559566787 || TP : 0.5694444444444444 || TN : 0.46616541353383456
Score : 0.49458483754512633 || TP : 0.5194805194805194 || TN : 0.4634146341463415
Score : 0.5054151624548736 || TP : 0.5465116279069767 || TN : 0.4380952380952381
Score : 0.48375451263537905 || TP : 0.5136986301369864 || TN : 0.45038167938931295
Score : 0.4693140794223827 || TP : 0.4722222222222222 || TN : 0.4639175257731959
Score : 0.48736462093862815 || TP : 0.525 || TN : 0.4358974358974359
Score : 0.51985559566787 || TP : 0.5280898876404494 || TN : 0.5050505050505051
Score : 0.49458483754512633 || TP : 0.5611510791366906 || TN : 0.427536231884058
Score : 0.49458483754512633 || TP : 0.5384615384615384 || TN : 0.4380165289256198
Score : 0.4657039711191336 || TP : 0.4899328859060403 || TN : 0.4375
Score : 0.4693140794223827 || TP : 0.5205479452054794 || TN : 0.4122137404580153
Score : 0.5306859205776173 || TP : 0.5714285714285714 || TN : 0.46788990825688076
Score : 0.4584837545126354 || TP : 0.5 || TN : 0.4

In [25]:
print("Random Forest score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

Random Forest score moyen : 0.5040851225536767 || TP moyen : 0.538149033546309 || TN moyen : 0.4600308310035195


In [26]:
scores,TPs,TNs = cross_validation_loop(predict_data,"MLP")

Score : 0.51985559566787 || TP : 0.5675675675675675 || TN : 0.46511627906976744
Score : 0.48014440433212996 || TP : 0.5068493150684932 || TN : 0.45038167938931295
Score : 0.5126353790613718 || TP : 0.56 || TN : 0.4566929133858268
Score : 0.5234657039711191 || TP : 0.5496688741721855 || TN : 0.49206349206349204
Score : 0.4657039711191336 || TP : 0.46308724832214765 || TN : 0.46875
Score : 0.5342960288808665 || TP : 0.5734265734265734 || TN : 0.4925373134328358
Score : 0.5270758122743683 || TP : 0.541095890410959 || TN : 0.5114503816793893
Score : 0.5054151624548736 || TP : 0.5657894736842105 || TN : 0.432
Score : 0.48014440433212996 || TP : 0.5266666666666666 || TN : 0.4251968503937008
Score : 0.5090252707581228 || TP : 0.5302013422818792 || TN : 0.484375
Score : 0.5126353790613718 || TP : 0.56 || TN : 0.4566929133858268
Score : 0.5451263537906137 || TP : 0.5933333333333334 || TN : 0.4881889763779528
Score : 0.4729241877256318 || TP : 0.5131578947368421 || TN : 0.424
Score : 0.534296028

In [27]:
print("MLP score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

MLP score moyen : 0.5078852365570967 || TP moyen : 0.5443692036536304 || TN moyen : 0.4653102613033886


### Using TensorFlow

In [13]:
%load_ext tensorboard

In [14]:
!rm -rf ./logs/

In [15]:
shape = (len(predict_data.columns)-3,)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=shape),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [16]:
print(train_data.shape, test_data.shape)

(178341, 63) (44586, 63)


In [17]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', 'Precision', 'Recall'])

In [18]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [19]:
model.fit(train_data, train_labels, epochs=10, validation_data=(test_data,test_labels),callbacks=[tensorboard_callback])

Epoch 1/10
5574/5574 [==============================] - 421s 75ms/step - loss: 0.6776 - accuracy: 0.5739 - precision: 0.5736 - recall: 0.5719 - val_loss: 0.6664 - val_accuracy: 0.5938 - val_precision: 0.5993 - val_recall: 0.5643
Epoch 2/10
 273/5574 [>.............................] - ETA: 5:57 - loss: 0.6669 - accuracy: 0.5928 - precision: 0.5995 - recall: 0.5509

KeyboardInterrupt: 

In [30]:
time = str(datetime.date.today())
#model.save('./models/1m_200_2000_200_'+time)

INFO:tensorflow:Assets written to: ./models/1m_200_2000_200_2022-09-29\assets


In [32]:
loaded_model = tf.keras.models.load_model('./models/1m_200_2000_200_'+time)

In [33]:
test_loss, test_acc = loaded_model.evaluate(test_data,  test_labels, verbose=2)

#print('\nTest accuracy:', test_acc)

3344/3344 - 21s - loss: 3.0073 - accuracy: 0.6279 - precision: 0.6242 - recall: 0.6940 - 21s/epoch - 6ms/step


ValueError: too many values to unpack (expected 2)

In [50]:
test_labels.value_counts()

1    55000
0    52000
Name: Target, dtype: int64

In [24]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])



In [59]:
#preds = np.argmax(probability_model.predict(test_data),axis=1)
preds = np.where(model.predict(test_data)>0.5, 1, 0)

In [60]:
np.unique(preds, return_counts=True)

(array([0, 1]), array([45852, 61148], dtype=int64))

In [17]:
conf = confusion_matrix(test_labels,preds)
conf

NameError: name 'confusion_matrix' is not defined

In [62]:
TN = conf[0,0] / (conf[0,0] + conf[1,0])
TN

0.6329276803629067

In [63]:
TP = conf[1,1] / (conf[1,1] + conf[0,1])
TP

0.6242068424151239

# Predicting 2021 with 2019 and 2020

In [13]:
data_19_20 = predict_data.loc[(predict_data["Unix"] >= 1546300800) & (predict_data["Unix"] < 1609459200)]

In [14]:
data_19_20

,Unix,Date,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,Vol-1,RSI-1,MACD-1,MACD_H-1,Variation-2,Vol-2,RSI-2,MACD-2,MACD_H-2,Variation-3,Vol-3,RSI-3,MACD-3,MACD_H-3,Variation-4,Vol-4,RSI-4,MACD-4,MACD_H-4,Variation-5,Vol-5,RSI-5,MACD-5,MACD_H-5,Variation-6,Vol-6,RSI-6,MACD-6,MACD_H-6,...,Vol-12,RSI-12,MACD-12,MACD_H-12,Variation-13,Vol-13,RSI-13,MACD-13,MACD_H-13,Variation-14,Vol-14,RSI-14,MACD-14,MACD_H-14,Variation-15,Vol-15,RSI-15,MACD-15,MACD_H-15,Variation-16,Vol-16,RSI-16,MACD-16,MACD_H-16,Variation-17,Vol-17,RSI-17,MACD-17,MACD_H-17,Variation-18,Vol-18,RSI-18,MACD-18,MACD_H-18,Variation-19,Vol-19,RSI-19,MACD-19,MACD_H-19,Target
1051100,1546301340,2019-01-01 00:09:00,-0.329127,-0.472447,-0.466916,-0.006785,0.022812,-0.278840,-0.163178,0.120873,-0.081105,-0.267165,-0.669165,-0.020851,-0.018919,0.062605,-0.333742,-0.531060,-0.029239,-0.037302,0.110160,-0.320416,-0.190011,-0.031282,-0.035239,0.180212,-0.307819,0.112435,-0.025670,-0.013362,-0.025324,-0.346650,0.053628,-0.021920,-0.000780,0.095597,-0.343283,0.529031,-0.015081,0.017446,...,-0.210484,3.242450,13.828973,24.956414,0.091844,-0.209404,3.242450,15.345395,23.972343,-0.192876,-0.348678,3.239983,16.351304,21.836032,-0.000855,-0.347596,3.239983,16.953059,19.058990,-0.000855,-0.346863,3.239983,17.231312,15.982488,-0.195007,-0.348678,3.237490,17.245581,12.823696,-0.000855,-0.342478,3.237843,17.060219,9.769129,0.306570,-0.335097,3.237989,16.732201,6.948501,1
1051120,1546302540,2019-01-01 00:29:00,-0.339529,-0.000855,-1.274979,-0.034384,-0.084242,-0.156757,-0.163178,2.124061,-0.000855,-0.348348,-1.604573,-0.040456,-0.083440,-0.024858,-0.348678,-1.530843,-0.045673,-0.080539,-0.000855,-0.345722,-1.530843,-0.049250,-0.073884,-0.170817,-0.346573,-1.809160,-0.057949,-0.082094,0.169270,-0.348678,-0.934083,-0.057667,-0.064931,-0.000855,-0.346812,-1.248113,-0.056799,-0.049652,...,-0.348678,-0.216243,-0.041032,0.011901,-0.000855,-0.345954,-0.216243,-0.040641,0.010553,0.137914,-0.340611,0.475876,-0.034619,0.024357,0.196340,-0.348678,1.088781,-0.022031,0.052748,-0.000855,-0.348678,1.207080,-0.011928,0.068896,-0.000855,-0.348678,1.207080,-0.003887,0.076365,-0.000855,-0.348678,2.277658,0.002447,0.077830,-0.000855,-0.283465,1.965031,0.007371,0.075277,1
1051140,1546303740,2019-01-01 00:49:00,-0.325023,-0.253338,0.909922,0.059382,0.033833,-0.266360,-0.163178,-0.416507,0.096356,-0.316663,1.094980,0.056345,0.019042,-0.175547,-0.348678,0.209299,0.046667,-0.010343,-0.000855,-0.348208,0.284454,0.038541,-0.029746,-0.000855,-0.326464,0.358724,0.031726,-0.041807,0.023099,-0.348678,0.595460,0.026925,-0.046132,-0.000855,-0.348121,0.145624,0.022847,-0.047683,...,-0.348678,-1.152155,0.006782,-0.038910,-0.000855,-0.272475,-1.152155,0.005821,-0.033668,-0.062648,-0.302550,-0.545211,0.002642,-0.035334,0.138586,-0.338360,-0.310238,0.005411,-0.020951,-0.069324,-0.250277,0.220905,0.004904,-0.018100,-0.148383,-0.265816,-0.474624,-0.001168,-0.030527,-0.246781,-0.114070,-1.163170,-0.015265,-0.061674,-0.000855,-0.306172,-1.274979,-0.026146,-0.078093,1
1051160,1546304940,2019-01-01 01:09:00,-0.210135,-0.132709,0.199841,0.024788,-0.031765,-0.151874,-0.163178,0.025949,-0.084825,-0.223822,-0.244013,0.015480,-0.050009,0.140761,-0.303221,-0.034856,0.013373,-0.045575,0.074453,-0.257242,0.291677,0.014419,-0.033696,-0.038253,-0.336800,0.285136,0.013643,-0.029007,-0.051689,-0.348678,-0.310548,0.010940,-0.030351,-0.000855,-0.348432,-0.522989,0.008686,-0.030235,...,-0.325771,0.475583,0.017767,0.009196,0.139524,-0.345920,0.828898,0.026676,0.030900,-0.049207,-0.266947,1.158432,0.031503,0.037475,0.220858,-0.348678,1.875038,0.043348,0.061279,-0.000855,-0.348678,1.683387,0.052123,0.072213,-0.000855,-0.322905,1.557913,0.058394,0.074342,0.026884,-0.348678,1.799420,0.063688,0.073463,-0.000855,-0.341105,2.088142,0.067100,0.067785,1
1051180,1546306140,2019-01-01 01:29:00,-0.348357,-0.317363,1.156960,0.084504,0.031952,-0.296745,-0.163178,1.617640,-0.092476,-0.348678,0.595365,0.073322,-0.003987,-0.000855,-0.348678,0.59

In [15]:
data_2021 = predict_data.loc[predict_data["Unix"] >= 1609459200]

In [16]:
data_2021

,Unix,Date,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,Vol-1,RSI-1,MACD-1,MACD_H-1,Variation-2,Vol-2,RSI-2,MACD-2,MACD_H-2,Variation-3,Vol-3,RSI-3,MACD-3,MACD_H-3,Variation-4,Vol-4,RSI-4,MACD-4,MACD_H-4,Variation-5,Vol-5,RSI-5,MACD-5,MACD_H-5,Variation-6,Vol-6,RSI-6,MACD-6,MACD_H-6,...,Vol-12,RSI-12,MACD-12,MACD_H-12,Variation-13,Vol-13,RSI-13,MACD-13,MACD_H-13,Variation-14,Vol-14,RSI-14,MACD-14,MACD_H-14,Variation-15,Vol-15,RSI-15,MACD-15,MACD_H-15,Variation-16,Vol-16,RSI-16,MACD-16,MACD_H-16,Variation-17,Vol-17,RSI-17,MACD-17,MACD_H-17,Variation-18,Vol-18,RSI-18,MACD-18,MACD_H-18,Variation-19,Vol-19,RSI-19,MACD-19,MACD_H-19,Target
2064420,1609459680,2021-01-01 00:08:00,-0.025056,0.022950,0.417542,-0.326229,1.055277,-0.296745,0.016964,0.051797,0.297738,-0.148565,1.022397,-0.136921,1.344475,0.041965,-0.082574,1.297912,0.025713,1.505349,0.012214,-0.196942,0.806634,0.156727,1.550490,0.278150,-0.141744,1.300388,0.340786,1.726775,0.017070,-0.035411,1.437312,0.486452,1.766349,-0.075473,0.084682,1.262523,0.572733,1.641080,...,0.743293,3.242280,142.804882,223.364022,-0.097894,1.468010,3.244261,152.254226,203.661526,0.080930,0.016519,3.244686,157.946879,177.972307,-0.171237,0.147181,3.243784,160.556058,149.272717,-0.092077,2.297348,3.243301,160.743352,119.913106,0.046214,-0.140457,3.243301,159.072290,91.514629,0.055845,0.048990,3.243431,155.967172,65.006292,-0.212085,-0.148241,3.242313,151.693925,40.712789,0
2064440,1609460880,2021-01-01 00:28:00,1.235547,-0.067007,-1.119984,-0.958632,-0.054895,-0.296745,0.140653,0.580934,0.087136,0.590014,-0.845958,-0.955965,-0.036870,0.034851,2.994560,-0.917234,-0.932375,0.032842,0.073664,-0.164294,-0.509659,-0.881114,0.161733,-0.154789,0.023824,-0.725001,-0.876723,0.140989,0.073330,-0.024904,-0.731635,-0.841248,0.206535,-0.184985,-0.105216,-1.236472,-0.858640,0.119269,...,-0.183870,-0.515060,-0.774573,0.312876,-0.416947,-0.144609,-0.728193,-0.852356,0.044756,0.056217,1.563090,-0.551666,-0.886675,-0.054885,-0.146590,1.640388,-0.868890,-0.946685,-0.202488,0.541444,-0.123238,-0.116417,-0.822243,0.166855,-0.067646,1.059105,-0.281343,-0.735235,0.363404,0.224667,-0.139513,0.157132,-0.591698,0.670025,0.075856,-0.010583,0.159882,-0.449689,0.911287,1
2064460,1609462080,2021-01-01 00:48:00,0.365838,-0.077123,-0.742743,-1.493196,0.504541,-0.296745,0.395642,1.314715,0.067571,3.039315,-0.085988,-1.423845,0.586897,-0.110258,0.339540,-0.610844,-1.385947,0.569664,-0.161734,1.869252,-1.072349,-1.388459,0.449094,0.238696,-0.127895,-0.923258,-1.303224,0.584510,-0.122605,-0.145849,-0.960070,-1.257936,0.587284,-0.037191,0.589914,-0.686703,-1.218965,0.572811,...,2.218792,-0.323585,-0.945622,0.641823,-0.009465,2.155923,-0.341421,-0.871350,0.709724,-0.292131,1.175464,-0.757652,-0.890110,0.518205,0.138111,0.007582,-0.574276,-0.853298,0.511842,-0.164831,0.528973,-0.663669,-0.863604,0.382239,-0.103320,2.129406,-0.567747,-0.892352,0.229824,0.114157,1.370075,-0.904186,-0.870496,0.241613,-0.240210,0.475204,-1.079694,-0.914725,0.076414,1
2064480,1609463280,2021-01-01 01:08:00,1.267819,0.094077,-1.009803,-1.210191,-1.555411,0.295218,-0.163178,0.896740,-0.351017,0.974089,-1.376763,-1.400169,-1.746354,0.094883,0.653805,-1.463986,-1.504382,-1.672472,0.039728,2.238974,-1.156977,-1.556732,-1.476314,-0.231420,1.550912,-1.803645,-1.649148,-1.425264,-0.384922,-0.040780,-2.016224,-1.817776,-1.585819,0.090936,-0.160539,-2.044293,-1.901760,-1.490587,...,-0.111265,-1.324809,-1.874168,-0.193178,-0.080923,0.040390,-1.256137,-1.867260,-0.136287,0.041302,0.224345,-1.364099,-1.827994,-0.005268,0.098931,1.590074,-0.801049,-1.746615,0.210834,-0.004452,-0.083957,-1.014166,-1.664036,0.386885,-0.216778,-0.172972,-1.329838,-1.644890,0.360102,0.060913,0.432634,-0.948714,-1.592725,0.425931,-0.001648,-0.153209,-0.369843,-1.533950,0.496058,1
2064500,1609464480,2021-01-01 01:28:00,0.943853,-0.260536,-0.195686,-0.035085,-0.258827,1.319082,-0.163178,-1.000253,-0.215084,1.990052,-0.431738,-0.163185,-0.545569,-0.138641,0.909504,-0.603004,-0.3

In [17]:
split_mode = "STATIC"

#if split_mode == "RAND":
#    train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.2, random_state=100)
if split_mode == "STATIC":
    train_data = data_19_20.drop(["Date", "Unix", "Target"],axis=1)
    train_labels = data_19_20["Target"]
    test_data = data_2021.drop(["Date", "Unix", "Target"],axis=1)
    test_labels = data_2021["Target"]

In [18]:
%load_ext tensorboard

In [19]:
shape = (len(test_data.columns),)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=shape),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [20]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', 'Precision', 'Recall'])

In [21]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [22]:
model.fit(train_data, train_labels, epochs=10, validation_data=(test_data,test_labels),callbacks=[tensorboard_callback], batch_size=64)

Epoch 1/10
792/792 [==============================] - 52s 62ms/step - loss: 0.3372 - accuracy: 0.8620 - precision: 0.8624 - recall: 0.8678 - val_loss: 0.6591 - val_accuracy: 0.8396 - val_precision: 0.8553 - val_recall: 0.8171
Epoch 2/10
792/792 [==============================] - 35s 44ms/step - loss: 0.2412 - accuracy: 0.8967 - precision: 0.8968 - recall: 0.9012 - val_loss: 0.3898 - val_accuracy: 0.8563 - val_precision: 0.8704 - val_recall: 0.8367
Epoch 3/10
792/792 [==============================] - 35s 44ms/step - loss: 0.2108 - accuracy: 0.9092 - precision: 0.9099 - recall: 0.9123 - val_loss: 0.4004 - val_accuracy: 0.8511 - val_precision: 0.8286 - val_recall: 0.8848
Epoch 4/10
792/792 [==============================] - 34s 43ms/step - loss: 0.1921 - accuracy: 0.9179 - precision: 0.9206 - recall: 0.9181 - val_loss: 0.4144 - val_accuracy: 0.8561 - val_precision: 0.8740 - val_recall: 0.8318
Epoch 5/10
792/792 [==============================] - 33s 41ms/step - loss: 0.1789 - accuracy: 0

In [23]:
test_loss, test_acc = model.evaluate(test_data,  test_labels, verbose=2)

#print('\nTest accuracy:', test_acc)

954/954 - 6s - loss: 0.5810 - accuracy: 0.8646 - precision: 0.8644 - recall: 0.8646 - 6s/epoch - 7ms/step


ValueError: too many values to unpack (expected 2)

In [25]:
time = str(datetime.date.today())
model.save('./models/20_to_6m_200_2000_200_y2019_2020_'+time)

INFO:tensorflow:Assets written to: ./models/20_to_6m_200_2000_200_y2019_2020_2022-10-05\assets
